In [ ]:
#### Author : Maximiliano Lopez Salgado
#### First commit: 2023-05-10
#### Last commit: 2023-05-10
#### Description: This notebook is used to perform EDA on the bike data set

In [ ]:
# import ML libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import folium
from folium.plugins import HeatMap

In [ ]:
<center><h1>Exploratory Data Analysis</center></h1>